# Part 1. How to use Beautiful soup to scrape the table from website


This notebook will show you how to scrape postal codes of Canada from the wikipedia page as follow.  
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

## Step 1. Install necessary library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests
print("libraries installed")

libraries installed


## Step 2. Scrape the html text

In [3]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text # get the html text by requests
soup = bs(url,'lxml') # transform html text into beatiful soup object


## Step 3. Match the target table in html text

In [4]:
match = soup.table  # Note that ".table" will show the first table in the html text.  In this case, it is our target table.
# pinrt(match)

In [5]:
table =soup.find_all('table', class_="wikitable sortable")  # find all tables in the html which class is "wikitable sortable", it will return a list
print('There are {} tables in this url '.format(len(table))) # Use len() to count numbers of table in this list, In this case, only return one table

# you can use ".find" to find the atrribute of your target, in this case is table, and class that it is.
# class is a python function. so to make a difference, we have to add underscore affter "class"  

There are 1 tables in this url 


## Step 4. Retrieve the data and save as pandas dataframe (Raw Data of Table)

In [6]:
# little test for retrieve the table data 
    
soup_cont = table[0].find_all('tr') # soup_cont = soup contents. Use "tr" to cut data 

print(soup_cont[1]) # This is the first data of the table 
clean = soup_cont[1].find_all('td') # use "td" to get the data string

Postalcode = clean[0].text.strip("\n")   # use ".text" to clean the html code and strip() to remove "\n"
Borough = clean[1].text.strip("\n")
Neighborhood = clean[2].text.strip("\n")

print('This is the first row of data :',[Postalcode,Borough,Neighborhood])



<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
This is the first row of data : ['M1A', 'Not assigned', 'Not assigned']


In [7]:
# Create a dictionary to insert the data of table
d = {'PostalCode':[],'Borough':[],'Neighborhood':[]} 

# Use loop to get all of the data
for i in range(len(soup_cont)-1):  
    d['PostalCode'].append(soup_cont[i+1].find_all('td')[0].text.strip("\n"))
    d['Borough'].append(soup_cont[i+1].find_all('td')[1].text.strip("\n"))
    d['Neighborhood'].append(soup_cont[i+1].find_all('td')[2].text.strip("\n"))

# Transform the python dictionary to pandas dataframe
target_table = pd.DataFrame(data=d) 

# Exchange the columns location to meet the requirement
cols = ['PostalCode','Borough','Neighborhood']
target_table = target_table.loc[:,cols]

# This is unclear row data
target_table.head()


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Step 5. Clean the raw data to meet the requirement

- ### Drop all the rows for those area that is **not assigned** a **Borough**

In [8]:
clean_df = target_table[target_table.Borough != 'Not assigned']
clean_df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


- ### Assign **Borough** to **Neighborhood** for those area that is **not assigned** a **Neighborhood**

In [9]:
# Find that area with "Not assigned" Neiborhood
clean_df.loc[clean_df.Neighborhood == 'Not assigned']


,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned


In [10]:
# Replace Neighborhood with its Borough
clean_df[clean_df.Neighborhood == 'Not assigned'].index[0] #　Find target index
clean_df.loc[clean_df[clean_df.Neighborhood == 'Not assigned'].index[0],'Neighborhood'] = 'Queen\'s Park'
clean_df.head(3)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Step 6. Group the data based on its postalcode

In [11]:
data = clean_df
group_data = clean_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "[%s]" % ', '.join(x)) # use ".groupby" and ".apply" method to save the group data 
group_data = group_data.reset_index()  # This is an important process to flatten the tuple.
group_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


In [12]:
print('Part 1 Answer:' ,group_data.reset_index().shape[0])

Part 1 Answer: 103


# Part 2. Add longitude and latitude columns into dataframe

### Step 1. Insert csv data from project dataset. (I have uploaded to the IBM cloud, it's really convenient)

In [13]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
group_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]


### Step 2. Because I use 'PostalCode' and 'Borough' to groupby data which is completely meet the csv location data. I simply transfer columns to group_data

In [15]:
group_data['Latitude'] = df_data_1['Latitude']
group_data['Longitude'] = df_data_1['Longitude']
group_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


# Part 3. Explore Data with FourSquare

Explore and cluster the neighborhoods in Toronto. 
You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.
Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. 

## Step 1. Import libraries

In [16]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


!pip install folium
import folium # map rendering library

print('Libraries imported.')

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Libraries imported.


## Step 2. Use Folium to take a look of these data

In [17]:
CLIENT_ID = 'ICBFWD00UVFRVPE4H3PVDBUXPBSZNOVSHFGK3ZODKTN4BKQE' # your Foursquare ID
CLIENT_SECRET = 'MQPFXLRVCJ5I43LNSGSG0LSNRNKP31ZAJ4LTZLLSRTR2XOUX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ICBFWD00UVFRVPE4H3PVDBUXPBSZNOVSHFGK3ZODKTN4BKQE
CLIENT_SECRET:MQPFXLRVCJ5I43LNSGSG0LSNRNKP31ZAJ4LTZLLSRTR2XOUX


In [18]:
# create map of Canada using latitude and longitude values
canada = group_data
canada.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


In [19]:
# canada['Latitude']
# canada['Longitude']
# canada['Neighborhood']
# canada['Borough']

In [20]:
# Find the center of target Postalcode
latitude = canada.Latitude.mean()
longitude = canada.Longitude.mean()

# Create a folium map object
map_canada = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add markers to map
for lat, lng, borough, neighborhood in zip(canada['Latitude'], canada['Longitude'], canada['Borough'], canada['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#008000',
        fill=True,
        fill_color='#008000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    


## Step 3. Define Foursquare Credentials and Version

In [21]:
CLIENT_ID = 'ICBFWD00UVFRVPE4H3PVDBUXPBSZNOVSHFGK3ZODKTN4BKQE' # your Foursquare ID
CLIENT_SECRET = 'MQPFXLRVCJ5I43LNSGSG0LSNRNKP31ZAJ4LTZLLSRTR2XOUX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ICBFWD00UVFRVPE4H3PVDBUXPBSZNOVSHFGK3ZODKTN4BKQE
CLIENT_SECRET:MQPFXLRVCJ5I43LNSGSG0LSNRNKP31ZAJ4LTZLLSRTR2XOUX


## Step 4. Let's explore the 30 top venues in the fist postalcode of the table within a radius of 500 meters. 

In [22]:
canada.head(1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353


In [23]:
# The fist postalcode Latitude 
print(canada.loc[0][3])
# The fist postalcode Longitude 
print(canada.loc[0][4])



43.8066863
-79.1943534


In [24]:
# type your answer here
radius = 500
LIMIT = 30
latitude = canada.loc[0][3]
longitude = canada.loc[0][4]
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)


In [25]:
# Request the data
results = requests.get(url).json()


In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
# retrive the venues data that we explore
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


## Step 5. Explore 30 top venues in all of postalcode within a radius of 500 meters. 

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
canada_venues = getNearbyVenues(names=canada['Neighborhood'],
                                   latitudes=canada['Latitude'],
                                   longitudes=canada['Longitude']
                                  )

# canada['Latitude']
# canada['Longitude']
# canada['Neighborhood']
# canada['Borough']

[Rouge, Malvern]
[Highland Creek, Rouge Hill, Port Union]
[Guildwood, Morningside, West Hill]
[Woburn]
[Cedarbrae]
[Scarborough Village]
[East Birchmount Park, Ionview, Kennedy Park]
[Clairlea, Golden Mile, Oakridge]
[Cliffcrest, Cliffside, Scarborough Village West]
[Birch Cliff, Cliffside West]
[Dorset Park, Scarborough Town Centre, Wexford Heights]
[Maryvale, Wexford]
[Agincourt]
[Clarks Corners, Sullivan, Tam O'Shanter]
[Agincourt North, L'Amoreaux East, Milliken, Steeles East]
[L'Amoreaux West]
[Upper Rouge]
[Hillcrest Village]
[Fairview, Henry Farm, Oriole]
[Bayview Village]
[Silver Hills, York Mills]
[Newtonbrook, Willowdale]
[Willowdale South]
[York Mills West]
[Willowdale West]
[Parkwoods]
[Don Mills North]
[Flemingdon Park, Don Mills South]
[Bathurst Manor, Downsview North, Wilson Heights]
[Northwood Park, York University]
[CFB Toronto, Downsview East]
[Downsview West]
[Downsview Central]
[Downsview Northwest]
[Victoria Village]
[Woodbine Gardens, Parkview Hill]
[Woodbine Heig

In [30]:
print(canada_venues.shape)
canada_venues.head()

(1331, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"[Rouge, Malvern]",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"[Rouge, Malvern]",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


### Let's check how many venues were returned for each Neighborhood

In [31]:
canada_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"[Adelaide, King, Richmond]",30,30,30,30,30,30
"[Agincourt North, L'Amoreaux East, Milliken, Steeles East]",3,3,3,3,3,3
[Agincourt],4,4,4,4,4,4
"[Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown]",11,11,11,11,11,11
"[Alderwood, Long Branch]",10,10,10,10,10,10
"[Bathurst Manor, Downsview North, Wilson Heights]",18,18,18,18,18,18
[Bayview Village],4,4,4,4,4,4
"[Bedford Park, Lawrence Manor East]",25,25,25,25,25,25
[Berczy Park],30,30,30,30,30,30


### Let's find out how many unique categories can be curated from all the returned venues

In [32]:
print('There are {} uniques categories.'.format(len(canada_venues['Venue Category'].unique())))

There are 241 uniques categories.


### Analyze Each Neighborhood

In [33]:
# one hot encoding
canada_onehot = pd.get_dummies(canada_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
canada_onehot['Neighborhood'] = canada_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [canada_onehot.columns[-1]] + list(canada_onehot.columns[:-1])
canada_onehot = canada_onehot[fixed_columns]



canada_onehot[['Yoga Studio','Neighborhood']] = canada_onehot[['Neighborhood','Yoga Studio']]

canada_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"[Rouge, Malvern]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[Rouge, Malvern]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [34]:
canada_onehot.rename(columns = {'Yoga Studio':'Neighborhood','Neighborhood':'Yoga Studio'}, inplace=1)
canada_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Yoga Studio,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"[Rouge, Malvern]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"[Rouge, Malvern]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [35]:
canada_onehot.shape

(1331, 241)

### Let's print each neighborhood along with the top 5 most common venues

In [36]:
canada_grouped = canada_onehot.groupby('Neighborhood').mean().reset_index()
canada_grouped


num_top_venues = 5

for hood in canada_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = canada_grouped[canada_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----[Adelaide, King, Richmond]----
              venue  freq
0              Café  0.10
1        Steakhouse  0.10
2  Asian Restaurant  0.07
3       Pizza Place  0.07
4             Hotel  0.07


----[Agincourt North, L'Amoreaux East, Milliken, Steeles East]----
               venue  freq
0         Playground  0.33
1   Asian Restaurant  0.33
2               Park  0.33
3  Accessories Store  0.00
4  Mobile Phone Shop  0.00


----[Agincourt]----
                venue  freq
0              Lounge  0.25
1      Breakfast Spot  0.25
2  Chinese Restaurant  0.25
3      Sandwich Place  0.25
4   Accessories Store  0.00


----[Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown]----
                 venue  freq
0        Grocery Store  0.18
1          Pizza Place  0.09
2  Fried Chicken Joint  0.09
3          Coffee Shop  0.09
4       Sandwich Place  0.09


----[Alderwood, Long Branch]----
          venue  freq
0   Pizza Place   0.2
1  Skating Ri

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = canada_grouped['Neighborhood']

for ind in np.arange(canada_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(canada_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"[Adelaide, King, Richmond]",Café,Steakhouse,Asian Restaurant,Pizza Place,Hotel,Smoke Shop,Noodle House,Opera House,Concert Hall,Monument / Landmark
1,"[Agincourt North, L'Amoreaux East, Milliken, S...",Park,Playground,Asian Restaurant,College Stadium,Comfort Food Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
2,[Agincourt],Lounge,Breakfast Spot,Chinese Restaurant,Sandwich Place,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
3,"[Albion Gardens, Beaumond Heights, Humbergate,...",Grocery Store,Fried Chicken Joint,Coffee Shop,Beer Store,Fast Food Restaurant,Sandwich Place,Liquor Store,Pizza Place,Pharmacy,Video Store
4,"[Alderwood, Long Branch]",Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Gym,Pub,Pharmacy,Dance Studio,Pool,Curling Ice
5,"[Bathurst Manor, Downsview North, Wilson Heights]",Coffee Shop,Pizza Place,Diner,Sandwich Place,Bridal Shop,Restaurant,Deli / Bodega,Bank,Fast Food Restaurant,Supermarket
6,[Bayview Village],Chinese Restaurant,Bank,Japanese Restaurant,Café,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
7,"[Bedford Park, Lawrence Manor East]",Coffee Shop,Fast Food Restaurant,Italian Restaurant,Grocery Store,Restaurant,Indian Restaurant,Japanese Restaurant,Liquor Store,Sandwich Place,Pub
8,[Berczy Park],Seafood Restaurant,Beer Bar,Cocktail Bar,Farmers Market,Café,Concert Hall,Tea Room,Liquor Store,Bistro,Belgian Restaurant
9,"[Birch Cliff, Cliffside West]",College Stadium,General Entertainment,Café,Skating Rink,Women's Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


## Step 6. Cluster Neighborhoods

### Run *k*-means to cluster the neighborhood into 5 clusters.

In [49]:
canada_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Yoga Studio,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"[Adelaide, King, Richmond]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033333,0.000000,0.0

In [50]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

canada_grouped_clustering = canada_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(canada_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_.astype(int)
type(kmeans.labels_[0])

numpy.int32

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
canada_data = canada.drop(columns='PostalCode')
canada_merged = canada_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
canada_merged = canada_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
canada_merged # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353,1.0,Fast Food Restaurant,Print Shop,Women's Store,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
1,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Women's Store,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
2,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,1.0,Electronics Store,Rental Car Location,Spa,Breakfast Spot,Medical Center,Mexican Restaurant,Intersection,Tech Startup,Pizza Place,Construction & Landscaping
3,Scarborough,[Woburn],43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Convenience Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
4,Scarborough,[Cedarbrae],43.773136,-79.239476,1.0,Athletics & Sports,Lounge,Hakka Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bank,Dim Sum Restaurant,Dessert Shop
5,Scarborough,[Scarborough Village],43.744734,-79.239476,4.0,Playground,Women's Store,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
6,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029,1.0,Playground,Discount Store,Department Store,Coffee Shop,Women's Store,Curling Ice,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore
7,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577,1.0,Bus Line,Bakery,Bus Station,Soccer Field,Park,Intersection,Fast Food Restaurant,Metro Station,Dim Sum Restaurant,Dessert Shop
8,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476,0.0,Motel,American Restaurant,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
9,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848,1.0,College Stadium,General Entertainment,Café,Skating Rink,Women's Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


In [52]:
canada_merged = canada_merged.drop(index=[16,21,93])  # drop nan data

In [56]:
# becaause of nan data, the data type of ['Cluster Labels'] become float,which doesn't meet the requirement of Kmeans (str or int is suitable)

canada_merged['Cluster Labels'] = pd.to_numeric(canada_merged['Cluster Labels'], downcast='signed')  # change data to int without float
canada_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Women's Store,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
1,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497,0,Construction & Landscaping,Bar,Women's Store,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run
2,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711,1,Electronics Store,Rental Car Location,Spa,Breakfast Spot,Medical Center,Mexican Restaurant,Intersection,Tech Startup,Pizza Place,Construction & Landscaping
3,Scarborough,[Woburn],43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Convenience Store,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
4,Scarborough,[Cedarbrae],43.773136,-79.239476,1,Athletics & Sports,Lounge,Hakka Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bank,Dim Sum Restaurant,Dessert Shop


### visualize the resulting clusters

In [62]:
# create map
latitude = canada_merged['Latitude'].mean()
longitude = canada_merged['Longitude'].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(canada_merged['Latitude'], canada_merged['Longitude'], canada_merged['Neighborhood'], canada_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters